In [1]:
from sympy import *
import sympy as sym
import numpy as np
import pandas as pd
from itertools import combinations
import openpyxl
from itertools import product

In [2]:
var('a c d SCR_Market SCR_Couterp SCR_life SCR_Health SCR_Non_Life x '
    'y p_12 p_13 p_14 p_15 p_23 p_24 p_25 p_34 p_35 p_45 SCR eps_1 eps_2 eps_3 eps_4 eps_5 eps_6 eps_7 eps_8 eps_9 eps_10')
(a, c, d, SCR_Market, SCR_Couterp, SCR_life, SCR_Health, SCR_Non_Life,
 x,y, p_12, p_13, p_14, p_15, p_23, p_24, p_25, p_34, p_35,p_45, SCR,
 eps_1, eps_2, eps_3, eps_4, eps_5, eps_6, eps_7, eps_8, eps_9, eps_10)
W = Matrix([[SCR_Market, SCR_Couterp, SCR_life, SCR_Health, SCR_Non_Life]])
R = Matrix([[1, p_12, p_13, p_14, p_15],
            [p_12, 1, p_23, p_24, p_25],
            [p_13,p_23,1,p_34,p_35],
            [p_14,p_24,p_34,1,p_45],
            [p_15,p_25,p_35,p_45,1]])

In [3]:
Standard = W.multiply(R)
Standard = Standard.multiply(W.T)

Standard = sym.expand(Standard[0])
Standard = list((str(Standard).split(" + ")))

In [4]:
R_zmiany = Matrix([[1, p_12, p_13, p_14, p_15],
            [p_12, 1, p_23, p_24, p_25],
            [p_13,p_23,1,p_34,p_35],
            [p_14,p_24,p_34,1,p_45],
            [p_15,p_25,p_35,p_45,1]])

In [5]:
Zmiana = W.multiply(R_zmiany)
zmiana = Zmiana.multiply(W.T)
zmiana = sym.expand(zmiana[0])
zmiana = list((str(zmiana).split(" + ")))

In [19]:
def PodajeWynik(Standard, zmiana):
    for i in range(len(Standard)):
        if Standard[i] in zmiana:
            zmiana.remove(Standard[i])
        else:
            return "Zle porównanie"
            break
    wynik = ''
    for j in range(len(zmiana) + 1):
        if j < len(zmiana):
            wynik = wynik + zmiana[j] + " + "
        elif j == len(zmiana):
            wynik = f"SCR: WRW^T_Standard + {wynik}"
    return wynik

In [20]:
rodzaj_ryzyka = {
    0 : 'Rynkowe',
    1 : 'NZPK',
    2 : 'Życie',
    3: 'Zdrowie',
    4: 'non-Majątkowe'
}

In [21]:
pary_liczb = []
for i in range(len(np.array(R_zmiany))):
    for j in range(len(np.array(R_zmiany))):
        if i > j:
            pary_liczb.append((i, j))

In [22]:
z = [eps_1, eps_2, eps_3, eps_4, eps_5, eps_6, eps_7, eps_8, eps_9, eps_10]
R_zmiany = np.array(R_zmiany)

In [23]:
writer = pd.ExcelWriter('Badanie_zmian_FS.xlsx', engine='xlsxwriter')

In [26]:
for liczba_zmian in range(0,2):
    kombinacje_par = [set(i) for i in combinations(pary_liczb, liczba_zmian)]
    dodana_wartosc = []
    Wyniki_końcowe = []
    SCR = []
    for P in kombinacje_par:
        R_zmiany_copy = R_zmiany.copy()
        m = 0
        ryzyko_dla_x = []
        ryzyko_dla_y = []
        for p in P:
            x, y = p
            R_zmiany_copy[x][y] = R_zmiany_copy[x][y] + z[m]
            R_zmiany_copy[y][x] = R_zmiany_copy[y][x] + z[m]
            ryzyko_dla_x.append(rodzaj_ryzyka[x])
            ryzyko_dla_y.append(rodzaj_ryzyka[y])
            dodana_wartosc.append(z[m])
            m = m + 1
        Zmiana = W.multiply(R_zmiany_copy)
        zmiana = Zmiana.multiply(W.T)
        zmiana = sym.expand(zmiana[0])
        zmiana = list((str(zmiana).split(" + ")))
        SCR.append(PodajeWynik(Standard, zmiana))
        Wyniki_końcowe.append(list(zip(ryzyko_dla_x, ryzyko_dla_y, dodana_wartosc)))
    Ostateczny = pd.DataFrame(
        {  # "ID": ID,
            "Zmienione ryzyka": Wyniki_końcowe,
            "Wynik": SCR
        })

    Ostateczny.to_excel(writer, sheet_name=f"zmian {liczba_zmian}")
    print(f"Zapisałem tabele dla {liczba_zmian} zmian")
    del Ostateczny

writer.save()



Zapisałem tabele dla 0 zmian
  Zmienione ryzyka                   Wynik
0               []  SCR: WRW^T_Standard + 
Zapisałem tabele dla 1 zmian
                    Zmienione ryzyka  \
0           [(NZPK, Rynkowe, eps_1)]   
1          [(Życie, Rynkowe, eps_1)]   
2             [(Życie, NZPK, eps_1)]   
3        [(Zdrowie, Rynkowe, eps_1)]   
4           [(Zdrowie, NZPK, eps_1)]   
5          [(Zdrowie, Życie, eps_1)]   
6  [(non-Majątkowe, Rynkowe, eps_1)]   
7     [(non-Majątkowe, NZPK, eps_1)]   
8    [(non-Majątkowe, Życie, eps_1)]   
9  [(non-Majątkowe, Zdrowie, eps_1)]   

                                               Wynik  
0  SCR: WRW^T_Standard + 2*SCR_Couterp*SCR_Market...  
1  SCR: WRW^T_Standard + 2*SCR_Market*SCR_life*ep...  
2  SCR: WRW^T_Standard + 2*SCR_Couterp*SCR_life*e...  
3  SCR: WRW^T_Standard + 2*SCR_Health*SCR_Market*...  
4  SCR: WRW^T_Standard + 2*SCR_Couterp*SCR_Health...  
5  SCR: WRW^T_Standard + 2*SCR_Health*SCR_life*ep...  
6  SCR: WRW^T_Standard + 2*SCR